# Quicksort
This file contains the quicksort example from the slides. In this notebook, we show how to create progressively more parallel versions of the quicksort algorithm using HPX.

Note: This is a simlified qsort. It can only sort a vector of items that are unique. Duplicate items will cause infinite recursion.

In [1]:
#include <algorithm>
#include <cstdlib>
#include <vector>
#include <unistd.h>
#include <random>
#include <iostream>
#include <chrono>

## The Basic Quicksort Algorithm
This version uses the built-in C++ partitioning algorithm.

In [2]:
template <typename RandomIter >
void quick_sort (RandomIter first, RandomIter last)
{
    ptrdiff_t size = last - first ;
    if (size > 1) {
        RandomIter pivot = partition(first, last,
                [p=*(first + rand() % size)](auto v) { return v < p; });
        quick_sort(first, pivot);
        quick_sort(pivot, last);
    }
}

This cell contains a convenience method for printing out a vector.

In [3]:
#undef show_max
#define show_max 10

In [4]:
template<typename V>
std::ostream& operator<<(std::ostream& o,const std::vector<V>& v) {
    auto i = v.begin();
    o << '{';
    if(i != v.end()) o << *i++;
    int shown = 1;
    for(;i != v.end(); ++i) {
        o << ' ' << *i;
        shown++;
        if(shown >= show_max) {
            o << "...";
            break;
        }
    }
    o << '}';
    return o;
}

This cell generates a vector of items, shuffles them, then passes them to our quicksort routine.

In [5]:
#undef sort_max
#define sort_max 200000

In [6]:
#undef iterations
#define iterations 20

In [7]:
.expr
using namespace std::chrono;
std::vector<int> v;
for(int i=0;i<sort_max;i++) v.push_back(i);
std::cout << v << std::endl;
int seed = getpid();
std::shuffle(v.begin(), v.end(), std::default_random_engine(seed));
std::cout << v << std::endl;

auto start = high_resolution_clock::now();
for(int i=0;i<iterations;i++)
    quick_sort(v.begin(), v.end());
auto finish = high_resolution_clock::now();
auto duration = duration_cast<microseconds>(finish - start);
std::cout << "duration: " << (duration.count()*1e-6) << std::endl;

std::cout << v << std::endl;

{0 1 2 3 4 5 6 7 8 9...}
{159491 85145 42224 113823 165572 90036 154316 65614 151004 190798...}
duration: 2.22455
{0 1 2 3 4 5 6 7 8 9...}


In [8]:
#include <run_hpx.cpp>

## Parallelizing!

When parallelizing an algorithm, it doesn't make sense to parallelize very small pieces of work.

In [9]:
#undef threshold
#define threshold (sort_max/10)

Here we do a very simple parallelization. We launch the first recursive call in a thread, run the second one, then wait for the first one to finish.

In [10]:
// In C++ you can't redefine symbols, but you can redefine macros.
// Here we employ this technique to allow the redefinition of quicksort2.
// Each time you edit and recompile this cell, just increase the version
// number and everything should be fine.
#undef quick_sort2
#define quick_sort2 quick_sort2v1
template <typename RandomIter >
void quick_sort2 (RandomIter first, RandomIter last)
{
    ptrdiff_t size = last - first ;
    //std::cout << "size=" << size << std::endl;
    if (size > threshold) {
        RandomIter pivot = partition(first, last,
                [p=*(first + rand() % size)](auto v) { return v < p; });
        // launch this task asynchronously
        auto f = hpx::async(hpx::launch::async,[&](){ quick_sort2(first, pivot); });
        // run this task synchronously
        quick_sort2(pivot, last);
        // wait for the asynchronous task to finish
        f.wait();
    } else {
        quick_sort(first, last);
    }
}

Run the parallel quicksort

In [11]:
.expr
run_hpx([](){
using namespace std::chrono;
std::vector<int> v;
for(int i=0;i<sort_max;i++) v.push_back(i);
std::cout << v << std::endl;
int seed = getpid();
std::shuffle(v.begin(), v.end(), std::default_random_engine(seed));
std::cout << v << std::endl;

auto start = high_resolution_clock::now();
for(int i=0;i<iterations;i++)
    quick_sort2(v.begin(), v.end());
auto finish = high_resolution_clock::now();
auto duration = duration_cast<microseconds>(finish - start);
std::cout << "duration: " << (duration.count()*1e-6) << std::endl;

std::cout << v << std::endl;
});

{0 1 2 3 4 5 6 7 8 9...}
{66991 189844 47180 168935 169005 127512 119388 134447 96200 116101...}
duration: 2.5186
{0 1 2 3 4 5 6 7 8 9...}


## Even more Parallelism!
This time, we will parallelize the partitioning step as well.

In [12]:
#undef quick_sort3
#define quick_sort3 quick_sort3v1
template <typename RandomIter >
hpx::future<void> quick_sort3 (RandomIter first, RandomIter last)
{
    ptrdiff_t size = last - first ;
    if (size > threshold) {
        hpx::future<RandomIter> pivot = hpx::partition(hpx::execution::par(hpx::execution::task), first, last,
            [p = *(first+rand()%size)](auto v) { return v < p; });
        return pivot.then([=](auto pf) {
            auto pivot = pf.get();
            auto f = hpx::async([&](){ return quick_sort3(pivot, last); });
            return hpx::when_all(quick_sort3(first, pivot), f);
        });
    } else {
        quick_sort(first, last);
        return hpx::make_ready_future();
    }
}

Run the parallel quicksort

In [13]:
.expr
run_hpx([](){
using namespace std::chrono;
std::vector<int> v;
for(int i=0;i<sort_max;i++) v.push_back(i);
std::cout << v << std::endl;
int seed = getpid();
std::shuffle(v.begin(), v.end(), std::default_random_engine(seed));
std::cout << v << std::endl;

auto start = high_resolution_clock::now();
for(int i=0;i<iterations;i++)
    quick_sort3(v.begin(), v.end()).wait();
auto finish = high_resolution_clock::now();
auto duration = duration_cast<microseconds>(finish - start);
std::cout << "duration: " << (duration.count()*1e-6) << std::endl;

std::cout << v << std::endl;
});

{0 1 2 3 4 5 6 7 8 9...}
{31502 161342 147332 33498 110722 11036 194027 7553 102842 40929...}
duration: 0.676801
{0 1 2 3 4 5 6 7 8 9...}
